In [1]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from dotenv import load_dotenv
from tqdm import tqdm
import time
import re
import random
from io import StringIO


# load dot env
env_file_name = "D:/Kuliah/AITF/Project/generate-diploy/gemini.env"

#fetch api gemini
if load_dotenv(dotenv_path=env_file_name):
    print(f"File{env_file_name} berhasil dimuat..")
else:
    print(f"File {env_file_name} tidak berhasil dimuat..")
    
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError(f"API KEY gagal dimuat pada file {env_file_name}..")
    
    
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash-lite')
print(f"konfig berhasil")

FileD:/Kuliah/AITF/Project/generate-diploy/gemini.env berhasil dimuat..
konfig berhasil


d:\Kuliah\AITF\Project\dtp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -------------------------------------------
# 2️⃣ FUNGSI PEMBUATAN PROMPT
# -------------------------------------------
def build_prompt(row):
    """
    Membuat prompt deskriptif berdasarkan satu baris data PON TIK.
    """
    return f"""
    Berdasarkan data berikut:

    Area Fungsi: {row['Area Fungsi']}
    Okupasi ID: {row['Okupasi ID']}
    Okupasi EN: {row['Okupasi EN']}
    Level: {row['Level']}
    Definisi Area Fungsi Kunci: {row['Definisi Area Fungsi Kunci']}
    Definisi Okupasi: {row['Definisi Okupasi']}
    Lingkup Bidang Pekerjaan: {row['Lingkup Bidang Pekerjaan']}
    Tanggung Jawab: {row['Tanggung Jawab']}
    Wewenang: {row['Wewenang']}
    Persyaratan: {row['Persyaratan']}

    Buatkan 20 variasi profil *talenta sintetis* yang relevan dengan deskripsi di atas.
    Setiap profil harus memiliki atribut berikut:

    - Jenjang Pendidikan dengan isian jawaban singkat (misalnya: S1, S2, D3, SMA, SMK, dll)
    - Jurusan
    - Pelatihan Tema
    - Nama Pelatihan
    - Sertifikasi
    - Bidang Sertifikasi
    - Pekerjaan Posisi
    - Lama Bekerja bulan dan tahun (misalnya: 2 tahun 3 bulan)

    Beberapa profil (sekitar 40%) boleh memiliki data tidak lengkap (misalnya ada 2-3 kolom kosong). Namun, pastikan kolom Jenjang Pendidikan dan Jurusan tidak memiliki nilai yang kosong.
    Jika pekerjaan posisi tidak diketahui, maka data lama bekerja juga tidak diketahui.

    Buat output **CSV murni** tanpa teks lain, tanpa koma di dalam isi kolom.
    Jika isi kolom memerlukan koma, ganti dengan tanda titik (.) atau strip (-).
    Pastikan hanya ada 8 kolom sesuai header.
    Kolom: Jenjang Pendidikan,Jurusan,Pelatihan Tema,Nama Pelatihan,Sertifikasi,Bidang Sertifikasi,Pekerjaan Posisi,Lama Bekerja
    """

In [3]:
# -------------------------------------------
# 3️⃣ FUNGSI PEMANGGILAN GEMINI API
# -------------------------------------------
def generate_synthetic_profiles(prompt):
    """
    Menghasilkan output tabel CSV dari prompt yang diberikan.
    """
    try:
        response = model.generate_content(
            contents=prompt,
            generation_config=genai.types.GenerationConfig(temperature=0.9)
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

In [4]:
# -------------------------------------------
# 4️⃣ PIPELINE UTAMA
# -------------------------------------------
def create_synthetic_dataset(input_path, output_path):
    # Membaca dataset PON TIK
    df = pd.read_csv(input_path)
    all_results = []

    # Iterasi tiap baris dataset
    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Memproses baris dataset"):
        prompt = build_prompt(row)
        synthetic_csv = generate_synthetic_profiles(prompt)

        if synthetic_csv:
            try:
                # 🔹 Pembersihan hasil teks sebelum parsing CSV
                clean_text = synthetic_csv.strip()
                clean_text = re.sub(r'\n\s*\n', '\n', clean_text)  # hapus baris kosong ganda
                clean_text = re.sub(r',\s*\n', '\n', clean_text)   # hapus koma berlebih di akhir baris
                clean_text = re.sub(r'\|', ',', clean_text)        # ubah delimiter '|' ke ',' bila perlu
                clean_text = re.sub(r'"+', '"', clean_text)
                clean_text = re.sub(r',\s*,', ',', clean_text)

                clean_text = "\n".join(
                    [line for line in clean_text.splitlines() if re.match(r'^[A-Za-z0-9]', line)]
                )

                # 🔹 Parsing hasil bersih
                temp_df = pd.read_csv(
                    StringIO(clean_text),
                    on_bad_lines='skip',  
                    engine='python'  
                )

                expected_cols = [
                    "Jenjang Pendidikan", "Jurusan", "Pelatihan Tema", "Nama Pelatihan",
                    "Sertifikasi", "Bidang Sertifikasi", "Pekerjaan Posisi", "Lama Bekerja"
                ]

                for col in expected_cols:
                    if col not in temp_df.columns:
                        temp_df[col] = '-'

                # Reorder kolom sesuai urutan yang diharapkan
                temp_df = temp_df[expected_cols]

                # Ganti nilai NaN atau kosong dengan '-'
                temp_df = temp_df.fillna('-')

                mask_lama = temp_df["Pekerjaan Posisi"].str.contains(r'\d+\s*(tahun|bulan)', case=False, na=False)
                temp_df.loc[mask_lama, "Lama Bekerja"] = temp_df.loc[mask_lama, "Pekerjaan Posisi"]
                temp_df.loc[mask_lama, "Pekerjaan Posisi"] = "-"

                # Tambahkan kolom referensi dari baris sumber
                temp_df["Okupasi ID"] = row["Okupasi ID"]
                temp_df["Area Fungsi"] = row["Area Fungsi"]
                temp_df["Level"] = row["Level"]
                all_results.append(temp_df)

            except Exception as e:
                print(f"Parsing error di baris {idx}: {e}")
                print("Contoh output mentah dari Gemini:")
                print(synthetic_csv[:300])  # tampilkan sebagian untuk debug
                continue

        # jeda agar tidak overload API
        time.sleep(random.uniform(1.5, 3.5))

    # Gabungkan semua hasil menjadi satu DataFrame
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_df.to_csv(output_path, index=False)
        print(f"✅ Dataset sintetis berhasil dibuat dan disimpan di: {output_path}")
    else:
        print("⚠️ Tidak ada hasil yang berhasil diproses.")

In [5]:
# -------------------------------------------
# 5️⃣ EKSEKUSI PIPELINE
# -------------------------------------------
import warnings
warnings.filterwarnings("ignore", message="This pattern is interpreted as a regular expression")

if __name__ == "__main__":
    create_synthetic_dataset(input_path="pon_tik_combined.csv",output_path="generated-dataset7.csv")

Memproses baris dataset:   0%|          | 1/255 [00:04<19:05,  4.51s/it]

Parsing error di baris 0: No columns to parse from file
Contoh output mentah dari Gemini:
"Jenjang Pendidikan","Jurusan","Pelatihan Tema","Nama Pelatihan","Sertifikasi","Bidang Sertifikasi","Pekerjaan Posisi","Lama Bekerja"
"S3","Hukum TI","Kepatuhan TIK","Manajemen Kepatuhan Teknologi Informasi","CGEIT","Tata Kelola TI","Chief Information Officer","10 tahun 0 bulan"
"S2","Manajemen TI",


Memproses baris dataset:   2%|▏         | 6/255 [00:42<28:06,  6.77s/it]

Parsing error di baris 5: No columns to parse from file
Contoh output mentah dari Gemini:
"S2","Teknik Informatika","Arsitektur Perusahaan","Certified Enterprise Architect Program","TOGAF Certification","Enterprise Architecture","Kepala Arsitektur Perusahaan","5 tahun 0 bulan"
"S1","Sistem Informasi","Manajemen Proyek TI","IT Project Management Professional","PMP","Project Management","M


Memproses baris dataset:   7%|▋         | 17/255 [02:17<31:04,  7.83s/it]

...

Parsing error di baris 229: No columns to parse from file
Contoh output mentah dari Gemini:
"S2","Teknik Informatika","Manajemen Layanan TI","ITIL Foundation","ITIL Foundation","Manajemen Layanan TI","KEPALA MANAJEMEN LAYANAN TI","12 tahun 0 bulan"
"S2","Manajemen Bisnis","Manajemen Proyek TI","PMP","Manajemen Proyek","Manajemen Proyek TI","LEAD IT SERVICES MANAGEMENT","15 tahun 6 bulan"
"


Memproses baris dataset:  93%|█████████▎| 238/255 [31:31<01:49,  6.43s/it]

Parsing error di baris 237: No columns to parse from file
Contoh output mentah dari Gemini:
"S1","Teknik Informatika","Pengembangan Web","Kursus Lengkap HTML CSS JavaScript","Web Developer Certification","Pengembangan Web","Pengelola Web","3 tahun 6 bulan"
"S1","Ilmu Komputer","Manajemen Konten","Pelatihan WordPress dan Elementor","Certified WordPress Specialist","Manajemen Konten","Pengel


Memproses baris dataset: 100%|██████████| 255/255 [33:29<00:00,  7.88s/it]

✅ Dataset sintetis berhasil dibuat dan disimpan di: generated-dataset7.csv
